In [1]:
import os
from dotenv import load_dotenv

_ = load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [2]:
# listing all available models
import google.generativeai as genai
for model in genai.list_models():
    print(model.name)

d:\AAAAAAA\RAG\Projects\LangChain - Basics, Chat and RAG\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gem

In [3]:
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

* `Connect Pinecone`

In [4]:
# Initialize Pinecone
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index_name = "quickstart"

try:
    pc.create_index(
        name=index_name,
        dimension=768,  # Google embedding-001 uses 768 dimensions
        metric="cosine",
        spec=pinecone.ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
except:
    pass

index = pc.Index(index_name)


* `Splitting & Embedding Text`

In [5]:
# Read Churchill speech
try:
    with open("../assets/churchill_speech.txt") as f:
        texts = f.read()
except FileNotFoundError:
    raise FileNotFoundError("churchill_speech.txt not found in ../assets/. Please verify the file path.")

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)
chunks = text_splitter.create_documents([texts])

print(f'Number of Chunks is {len(chunks)}')
print(chunks[2].page_content)

Number of Chunks is 300
From the moment that the French defenses at Sedan and on the Meuse were broken at the end of the


In [6]:
# Calculate embedding cost (approximate, as Google pricing differs)
enc = tiktoken.get_encoding("cl100k_base")  # Use a general encoding for token counting
total_tokens = sum([len(enc.encode(i.page_content)) for i in chunks])

print(f'Total Tokens is: {total_tokens}')
# Note: Google embedding pricing varies; check Google Cloud pricing for exact cost
print(f'Approximate token count for cost estimation: {total_tokens}')


Total Tokens is: 4820
Approximate token count for cost estimation: 4820


In [7]:
# Initialize Google embeddings with updated model
try:
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",  # Updated model name
        google_api_key=GOOGLE_API_KEY
    )
except Exception as e:
    raise Exception(f"Failed to initialize GoogleGenerativeAIEmbeddings: {str(e)}")

# Test embedding
try:
    vector_0 = embeddings.embed_query(chunks[0].page_content)
    print("Embedding successful. First 10 values:", vector_0[:10])
except Exception as e:
    raise Exception(f"Embedding failed: {str(e)}. Please verify your Google API key and Vertex AI API setup at https://console.cloud.google.com/apis/credentials.")

Embedding successful. First 10 values: [0.033345527946949005, -0.002517310669645667, -0.02232394367456436, -0.02673177421092987, 0.0484866239130497, 0.03872605413198471, 0.04826189577579498, -0.02397942915558815, 0.030039627104997635, 0.07229742407798767]


* `Upserting to Pincone`

In [8]:
try:
    vector_store = PineconeVectorStore.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=index_name,
        pinecone_api_key=PINECONE_API_KEY
    )
    print("Successfully upserted to Pinecone vector store.")
except Exception as e:
    raise Exception(f"Failed to upsert to Pinecone: {str(e)}. Please verify your Pinecone API key and index setup at https://app.pinecone.io.")

Successfully upserted to Pinecone vector store.


In [10]:
len(chunks)

300

In [11]:
# Check after
index.describe_index_stats() 

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 900}},
 'total_vector_count': 900,
 'vector_type': 'dense'}

In [13]:
query = 'Where should we fight?'
result = vector_store.similarity_search(query=query, k=3)

for r in result:
    print(r.page_content)
    print('-' * 50)

end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing
--------------------------------------------------
end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing
--------------------------------------------------
end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing
--------------------------------------------------


* `Answering using LLM (RAG)`

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest" , api_key=GOOGLE_API_KEY , temperature=0.2)

retriever = vector_store.as_retriever(search_type = "similarity" , search_kwargs={'k': 4})

chain = RetrievalQA.from_chain_type(llm = llm , chain_type="stuff" , retriever=retriever)

In [15]:
# An example
query = 'Answer only from the provided input. Where should we fight?'
answer = chain.invoke(query)
print(answer['result'])

France, the seas, and the oceans.


In [16]:
# Another example
query = 'Answer only from the provided input. Who was the king of Belgium at that time?'
answer = chain.invoke(query)
print(answer['result'])

This text mentions the King of the Belgians, but doesn't state his name.


In [17]:
# Another exmaple
query = 'Answer only from the provided input. Does French defenses at Sedan?'
answer = chain.invoke(query)
print(answer['result'])

The French defenses at Sedan and on the Meuse were broken.
